In [1]:
import kagglehub

kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
!pip install -q -U torch immutabledict sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [5]:
weights_dir = kagglehub.model_download(
    "google/gemma/pyTorch/2b-it"
)

Attaching model 'google/gemma/pyTorch/2b-it' to your Colab notebook...


In [6]:
import os

tokenizer_path= os.path.join(weights_dir, "tokenizer.model")

ckpt_path = os.path.join(weights_dir, "gemma-2b-it.ckpt")

In [7]:
!git clone https://github.com/google/gemma_pytorch.git

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 71 (delta 12), reused 8 (delta 8), pack-reused 55
Receiving objects: 100% (71/71), 2.13 MiB | 5.66 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [8]:
import sys

sys.path.append("gemma_pytorch")

In [9]:
from gemma_pytorch.gemma.config import get_config_for_2b
from gemma_pytorch.gemma.model import GemmaForCausalLM

In [10]:
import torch

model_config = get_config_for_2b()
model_config

GemmaConfig(vocab_size=256000, max_position_embeddings=8192, num_hidden_layers=18, num_attention_heads=8, num_key_value_heads=1, hidden_size=2048, intermediate_size=16384, head_dim=256, rms_norm_eps=1e-06, dtype='bfloat16', quant=False, tokenizer='tokenizer/tokenizer.model')

In [11]:
model_config.tokenizer = tokenizer_path

model_config.quant = "quant" in "2b-it"

In [12]:
torch.set_default_dtype(model_config.get_dtype())
device = torch.device("cuda")
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

In [13]:
model.generate(
    "Tell me a joke",
    device = device,
    output_len = 60,
)

" about a robot's name.\n\nWhat's the name of a robot named Rodney?\n\nRodney Asimov."

In [14]:
USER_CHAT_TEMPLATE = '<start_of_turn>user\n{prompt}<end_of_turn>\n'
MODEL_CHAT_TEMPLATE = '<start_of_turn>model\n{prompt}<end_of_turn>\n'

prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt="What is a good place for travel in the US?"
    )
    + MODEL_CHAT_TEMPLATE.format(prompt="New York")
    + USER_CHAT_TEMPLATE.format(
        prompt="What can I do in New York?"
    )
    + '<start_of_turn>model\n'
)

In [15]:
print("Chat prompt:\n", prompt)

Chat prompt:
 <start_of_turn>user
What is a good place for travel in the US?<end_of_turn>
<start_of_turn>model
New York<end_of_turn>
<start_of_turn>user
What can I do in New York?<end_of_turn>
<start_of_turn>model



In [17]:
model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device = device,
    output_len = 100,
)

"* **Visit the Statue of Liberty and Ellis Island:** Explore the iconic landmarks and learn about the history of immigration to the United States.\n* **Catch a Broadway show:** See a world-class production in New York City's vibrant theater scene.\n* **Explore Central Park:** Enjoy walking, biking, or picnicking in this sprawling green space.\n* **Visit the Metropolitan Museum of Art:** Immerse yourself in a world of art from different cultures.\n* **Take a"

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [X](http://x.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) 😎